In [1]:
import urllib
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier


In [2]:
for x in range(2013,2021):
    f = open(f"nov{x}-{x+1}.txt", "r")
    content = f.read()
    f.close()

    content = content.replace(" ","")

    g = open(f"nov{x}-{x+1}.txt", "w")
    g.write(content)
    g.close()

In [3]:
frames = []

for x in range(2013,2021):
    frames.append(pd.read_csv(f"nov{x}-{x+1}.txt", sep = ","))

data = pd.concat(frames)

In [4]:
data.sample(10)

,Aeg,Temperatuur,Niiskus,&Otilde;hur&otilde;hk,Tuulekiirus,Tuulesuund,Sademed,UVindeks,Valgustatus,Kiirgusvoog,Radioaktiivsus
48860,2020-04-2923:55:00,1.708857,76.153429,1010.542900,2.054298,31.787175,0.0,NaN,NaN,0.263415,NaN
54699,2019-05-1900:55:00,15.520183,56.053655,1017.789433,3.133050,125.688988,0.0,NaN,NaN,0.611572,NaN
24035,2017-02-0610:45:00,-9.408945,77.227780,1032.245733,6.725341,102.686837,0.0,NaN,NaN,207.445677,NaN
69204,2021-07-0422:15:00,23.488089,55.467452,1014.585633,2.461981,96.633236,0.0,NaN,NaN,0.000000,NaN
63315,2017-06-2221:05:00,14.958985,43.453376,1004.615000,4.603048,286.495311,0.0,NaN,NaN,78.805021,NaN
16005,2017-01-0913:35:00,-1.162365,91.240721,1018.145500,3.924947,243.819581,0.0,NaN,NaN,30.677537,NaN
20787,2021-01-1718:30:00,-10.010632,89.396645,1018.011433,3.726338,274.360841,0.0,NaN,NaN,6.179114,NaN
98165,2019-10-1623:05:00,8.898684,96.004921,1006.518000,1.022259,118.208322,0.0,NaN,NaN,NaN,NaN
80272,2014-10-2012:30:00,10.580218,99.394906,973.217455,3.514186,292.316024,0.0,NaN,NaN,NaN,NaN
81160,2016-08-2219:25:00,20.020826,84.501240,1020.109367,3.529111,284.663386,0.0,NaN,NaN,28.886399,NaN


In [5]:
data.shape

(811086, 11)

In [6]:
#f = open("kiirgusvoog_5a.csv", "r")
#content = f.read()
#f.close()

#content = content.replace(" ","")

#g = open("kiirgusvoog_5a_2.csv", "w")
#g.write(content)
#g.close()

#data = pd.read_csv("kiirgusvoog_5a_2.csv")

In [7]:
data = data.rename(columns = {'&Otilde;hur&otilde;hk':'Õhurõhk'})

In [8]:
def break_up_time(data):
    data["Aasta"]= data.apply(lambda row: row["Aeg"][0:4], axis=1)
    data["Kuu"] = data.apply(lambda row: row["Aeg"][5:7], axis=1)
    data["Päev"] = data.apply(lambda row: row["Aeg"][8:10], axis=1)
    data["Tund"] = data.apply(lambda row: row["Aeg"][10:12], axis=1)
    data["Minut"] = data.apply(lambda row: row["Aeg"][13:15], axis=1)
    data.Aasta = data.Aasta.astype('int')
    data.Kuu = data.Kuu.astype('int')
    data.Päev = data.Päev.astype('int')
    data.Tund = data.Tund.astype('int')
    data.Minut = data.Minut.astype('int')

In [9]:
break_up_time(data)
data.head()

,Aeg,Temperatuur,Niiskus,Õhurõhk,Tuulekiirus,Tuulesuund,Sademed,UVindeks,Valgustatus,Kiirgusvoog,Radioaktiivsus,Aasta,Kuu,Päev,Tund,Minut
0,2013-11-1500:00:00,2.253725,80.280623,1012.974940,1.396552,219.132229,0.0,NaN,NaN,NaN,0.097777,2013,11,15,0,0
1,2013-11-1500:05:00,2.269765,80.633583,1013.042850,1.246345,220.840096,0.0,NaN,NaN,NaN,0.134524,2013,11,15,0,5
2,2013-11-1500:10:00,2.259058,80.844891,1012.746232,1.387465,218.213758,0.0,NaN,NaN,NaN,0.091138,2013,11,15,0,10
3,2013-11-1500:15:00,2.222720,80.999738,1012.925478,1.803502,224.199997,0.0,NaN,NaN,NaN,0.097796,2013,11,15,0,15
4,2013-11-1500:20:00,2.135194,81.082619,1012.874290,1.640076,225.130637,0.0,NaN,NaN,NaN,0.096651,2013,11,15,0,20


In [10]:
data.columns

Index(['Aeg', 'Temperatuur', 'Niiskus', 'Õhurõhk', 'Tuulekiirus', 'Tuulesuund',
       'Sademed', 'UVindeks', 'Valgustatus', 'Kiirgusvoog', 'Radioaktiivsus',
       'Aasta', 'Kuu', 'Päev', 'Tund', 'Minut'],
      dtype='object')

In [11]:
data = data[["Aasta","Kuu","Päev","Tund","Minut","Temperatuur","Niiskus","Õhurõhk","Tuulekiirus","Tuulesuund","Sademed","Kiirgusvoog"]] # "Sadanudlumi"
#data['Sadanudlumi'] = data['Sadanudlumi'].fillna(0)
#data.columns = ["Aasta","Kuu","Päev","Tund","Minut","Temperatuur","Niiskus","Õhurõhk","Tuule kiirus","Tuule suund","Sademed","Kiirgusvoog"]
data['Sademed'] = data['Sademed'].fillna(0)
data['Tuulekiirus'] = data['Tuulekiirus'].fillna(method='backfill')
data['Tuulesuund'] = data['Tuulesuund'].fillna(method='backfill')
data['Temperatuur'] = data['Temperatuur'].fillna(method='backfill')
data['Niiskus'] = data['Niiskus'].fillna(method='backfill')
data['Õhurõhk'] = data['Õhurõhk'].fillna(method='backfill')
data['Kiirgusvoog'] = data['Kiirgusvoog'].fillna(method='backfill')
data["KiirgusvoogTulevikus"] = data["Kiirgusvoog"].shift(-1)
data = data.dropna()
data.isnull().sum()

Aasta                   0
Kuu                     0
Päev                    0
Tund                    0
Minut                   0
Temperatuur             0
Niiskus                 0
Õhurõhk                 0
Tuulekiirus             0
Tuulesuund              0
Sademed                 0
Kiirgusvoog             0
KiirgusvoogTulevikus    0
dtype: int64

In [12]:
data = data.rename(columns = {
    'Aasta':'year',
    'Kuu':'month',
    'Päev':'day',
    'Tund':'hour',
    'Minut':'minute',
    'Temperatuur':'temperature',
    'Niiskus':'humidity',
    'Õhurõhk':'atmospheric_pressure',
    'Tuulekiirus':'wind_speed',
    'Tuulesuund':'wind_direction',
    'Sademed':'precipitation',
    #'Sadanudlumi':'snow',
    'Kiirgusvoog':'radiation_flux',
    'KiirgusvoogTulevikus':'rad_flux_infuture'    
    })
data.columns

Index(['year', 'month', 'day', 'hour', 'minute', 'temperature', 'humidity',
       'atmospheric_pressure', 'wind_speed', 'wind_direction', 'precipitation',
       'radiation_flux', 'rad_flux_infuture'],
      dtype='object')

In [13]:
data[data.humidity == data.humidity.max()]

,year,month,day,hour,minute,temperature,humidity,atmospheric_pressure,wind_speed,wind_direction,precipitation,radiation_flux,rad_flux_infuture
51611,2014,5,13,11,0,8.624431,117.297204,990.984611,3.325451,290.185328,0.0,0.0,0.0
51611,2014,5,13,11,0,8.624431,117.297204,990.984611,3.325451,290.185328,0.0,0.0,0.0


In [14]:
data.describe()

,year,month,day,hour,minute,temperature,humidity,atmospheric_pressure,wind_speed,wind_direction,precipitation,radiation_flux,rad_flux_infuture
count,811085.000000,811085.000000,811085.000000,811085.000000,811085.000000,811085.000000,811085.000000,811085.000000,811085.000000,811085.000000,811085.000000,811085.000000,811085.000000
mean,2017.398950,6.494621,15.751352,11.481656,27.512518,6.825025,75.458473,1010.824081,3.258511,199.267411,0.004576,89.464947,89.464947
std,2.424143,3.526065,8.820120,6.926916,17.262782,8.905742,19.869568,11.040802,1.678265,91.412932,0.050455,181.339887,181.339887
min,2013.000000,1.000000,1.000000,0.000000,0.000000,-24.307660,0.000000,963.557524,0.000000,0.000000,0.000000,-2.625525,-2.625525
25%,2016.000000,3.000000,8.000000,5.000000,14.000000,0.635374,62.394290,1003.885773,2.053779,124.554193,0.000000,0.000000,0.000000
50%,2018.000000,6.000000,16.000000,11.000000,30.000000,5.843931,81.315327,1011.116467,3.025117,224.132079,0.000000,2.639460,2.639460
75%,2019.000000,10.000000,23.000000,17.000000,45.000000,13.633509,91.499743,1018.149000,4.240506,274.416061,0.000000,74.003317,74.003317
max,2021.000000,12.000000,31.000000,23.000000,59.000000,33.553939,117.297204,1045.714767,15.497177,360.000000,9.799000,1132.409778,1132.409778


In [15]:
data.sample(50)

,year,month,day,hour,minute,temperature,humidity,atmospheric_pressure,wind_speed,wind_direction,precipitation,radiation_flux,rad_flux_infuture
81122,2014,10,23,11,20,-5.751292,65.367296,1014.431032,4.687319,111.552990,0.0,0.000000,0.000000
16379,2018,1,10,20,55,-3.012333,93.883222,1024.423767,1.484302,285.301642,0.0,1.035408,1.019756
13811,2016,1,1,22,20,-14.731138,84.631379,1040.342767,4.937413,100.568888,0.0,0.000000,0.000000
73893,2014,9,28,1,30,7.890639,95.567576,1013.948115,0.592885,309.605725,0.0,0.000000,0.000000
55676,2017,5,27,8,30,11.159337,68.466480,1018.280433,0.772845,304.999555,0.0,460.396509,473.335399
50565,2014,5,9,19,50,11.710901,92.394435,1000.424293,3.866512,151.956270,0.0,0.000000,0.000000
68558,2020,7,7,9,25,14.899973,73.641694,1004.663133,4.849731,226.365228,0.0,284.581631,304.383100
29811,2014,2,26,13,0,0.736809,47.382404,1020.101794,2.155039,146.997070,0.0,0.000000,0.000000
85071,2018,9,6,10,10,18.189702,99.200108,1018.405316,2.470872,81.138009,0.0,204.390516,226.794419
78632,2017,8,15,1,30,13.533243,73.865232,1025.327167,1.270730,18.321001,0.0,0.000000,0.000000


In [16]:
data.head(30)

,year,month,day,hour,minute,temperature,humidity,atmospheric_pressure,wind_speed,wind_direction,precipitation,radiation_flux,rad_flux_infuture
0,2013,11,15,0,0,2.253725,80.280623,1012.974940,1.396552,219.132229,0.0,0.0,0.0
1,2013,11,15,0,5,2.269765,80.633583,1013.042850,1.246345,220.840096,0.0,0.0,0.0
2,2013,11,15,0,10,2.259058,80.844891,1012.746232,1.387465,218.213758,0.0,0.0,0.0
3,2013,11,15,0,15,2.222720,80.999738,1012.925478,1.803502,224.199997,0.0,0.0,0.0
4,2013,11,15,0,20,2.135194,81.082619,1012.874290,1.640076,225.130637,0.0,0.0,0.0
5,2013,11,15,0,25,2.075500,81.316279,1012.867377,1.369254,221.732026,0.0,0.0,0.0
6,2013,11,15,0,30,2.041885,81.697612,1012.920600,1.314422,223.314140,0.0,0.0,0.0
7,2013,11,15,0,35,2.038886,82.163594,1013.180392,1.228393,221.865827,0.0,0.0,0.0
8,2013,11,15,0,40,2.022422,82.593353,1013.001292,1.528448,219.857440,0.0,0.0,0.0
9,2013,11,15,0,45,2.020750,83.062526,1012.941358,1.403501,223.718700,0.0,0.0,0.0


In [17]:
data.to_csv("data.csv")

## trying to get data directly from web page

In [18]:
#url = 'https://meteo.physic.ut.ee/et/archive.php?do=data&begin%5Byear%5D=2016&begin%5Bmon%5D=11&begin%5Bmday%5D=18&end%5Byear%5D=2021&end%5Bmon%5D=11&end%5Bmday%5D=18&9=1&12=1&10=1&15=1&16=1&14=1&snow_16=1&ok=+Esita+päring+'


def get_weather_data(start_year, start_month, start_day, end_year, end_month, end_day):
    url = f'https://meteo.physic.ut.ee/et/archive.php?do=data&begin%5Byear%5D={start_year}&begin%5Bmon%5D={start_month}&begin%5Bmday%5D={start_day}&end%5Byear%5D={end_year}&end%5Bmon%5D={end_month}&end%5Bmday%5D={end_day}&9=1&12=1&11=1&14=1&ok=+Esita+p%C3%A4ring+'
    c = pd.read_csv(url)
    #c.columns = ['Aeg', 'Temperatuur', 'Niiskus', 'Valgustatus', 'Kiirgusvoog']
    return c

In [19]:
train = get_weather_data(2011, 1, 1, 2015, 12, 12)

In [20]:
train.head()

,Aeg,Temperatuur,Niiskus,Valgustatus,Kiirgusvoog
0,2011-01-01 00:00:00,-6.936343679417071,98.6784171261325,1354.9408873569798,
1,2011-01-01 00:05:00,-6.9568330698942695,98.59701577484581,1029.54357837049,
2,2011-01-01 00:10:00,-6.97364936736915,98.42287783684701,1418.1783680055298,
3,2011-01-01 00:15:00,-6.94325190753387,98.49124327899729,1294.6821264335601,
4,2011-01-01 00:20:00,-6.86547316481287,98.64064185206159,1374.1625144454501,


In [21]:
train.isnull().sum()

Aeg             0
 Temperatuur    0
 Niiskus        0
 Valgustatus    0
 Kiirgusvoog    0
dtype: int64